# Querying

## Introduction
> __Prometheus has significant querying capabilities (including its native PromQL language), which we will learn about in this lesson.__

- __Please initiate an instance of `node` and a Prometheus server (inside some docker containers) following the steps described in the previous lesson.__
- __Go to `localhost:9090` to access the query/expression prompt.__ 


## Data Types

Only one of the four listed below is allowed:
- `float64` (any scalar is a `float`)
- `string` (unused by `prometheus` currently)
- __instant vectors__
- __range vectors__

## Instant Vector Selectors

> __Instant vectors are created via selection based on some matching patterns (labels, metric names, etc.).__

__The easiest form is a simple `metric`-name statement that returns a vector of values.__

Input the following in the expressions field on your Prometheus dashboard. Some of these metrics might appear differently depending on your OS.

In [ ]:
node_cpu_seconds_total

In [ ]:
# Windows 
windows_cpu_time_total

![](images/prometheus_vector_dtype_basic.png)

We can further filter the metric by specific labels. Consider the example:

In [ ]:
node_cpu_seconds_total{cpu=~"0|1", instance!="localhost:8081", mode="user"}

In [ ]:
# on windows
windows_cpu_time_total{core=~"0,0|0,1", job="wmiexporter"}

![](images/prometheus_node_query_matching.png)

### Matching

> Prometheus supports different comparison operators __and regex matching__ (via [Google's RE2 syntax](https://github.com/google/re2/wiki/Syntax)).

*Comparison operators*
- `=`: labels that are equal.
- `!=`: labels that __are not__ equal.
- `=~`: labels that __regex match__ the string.
- `!~`: labels that __regex UNmatch__ the string.

Other than that, regex matching works normally (check out the specification (link above) when in doubt).

#### __name__

Similar to Python's `__name__`, Prometheus also provides this `label` as an internal label. __It allows us to match expressions__. 

As an example, a quick way to match strings is using the `".*"` expression. The `.` parameter signifies the matching of any character, and `*` signifies the matching of any number of characters afterwards. 

In [ ]:
{__name__=~"node.*_seconds"}

In [ ]:
{__name__=~"windows.*_time_.*"}

![](images/prometheus_name_match.png)

*Things to note*

> __Regex matching must match something, i.e. the expression must be valid.__


Consider the example of a label matching every job __and empty job,__ where the job starts with wmi:

In [ ]:
{job=~"wmi.*"}

Conversely, the `+` parameter will match any job that does not have an empty string: 

In [ ]:
{job=~".+"}

## Range Vector selectors

> __Range vectors are created by slicing the time series based on the duration.__

Previously, we used `{}` for instant selectors. This time, we will use `[]`: 

In [ ]:
node_cpu_seconds_total{cpu=~"0|1", mode="idle"}[20s]

In [ ]:
# Windows
windows_cpu_time_total{core=~"0,.*|0,.*", job="wmiexporter"}[20s]

> Range vectors __cannot be graphed.__ They must be transformed into instant vectors via a function. This is because __they have multiple values for a single timestamp.__

![](images/prometheus_range_selector.png)

Above, we can observe __the multidimensionally aggregated data__:
- We get `4` values for each time series.
- There are `4` values for a `20s` range __because the data are scrapped at `5s` intervals.__

Now, we should be able to perform an operation on the grouped data.

To plot the above, we can summarise the data using the following query (for example):

In [ ]:
rate(node_cpu_seconds_total{cpu=~"0|1", mode="idle"}[5m])

In [ ]:
rate(windows_cpu_time_total{core=~"0,.*|0,.*", mode="idle"}[5m])

![](images/prometheus_range_rate_cpus.png)

There are a few available time units:

- `ms`: milliseconds
- `s`: seconds
- `m`: minutes
- `h`: hours
- `d`: days (assuming a day always has 24 h)
- `w`: weeks (assuming a week always has 7 d)
- `y`: years (assuming a year always has 365 d)

> Although the units above can be mixed, __they must be ordered from the largest to the smallest,__ e.g. `1h30m` and `2d3h15m10s7ms`.

### Offsets

> __Offsets allow `jumps` to a specified point in time.__

For example, the query below could return a `5` minute rate of the `http_requests` we made `1` week ago:

In [ ]:
rate(http_requests_total[5m] offset 1w)

## Operators

> __Prometheus's language (PromQL) provides the standard set of operators (logical, arithmetic, etc.).__

| Arithmetic        | Comparison          | Logical  |
| ------------- |:-------------:| -----:|
| + | == | and |
| - | != | or |
| / | > | unless (complement) |
| \% | <  | |
| ^ | >=  | |
|  | <=   | |


These follow the standard broadcasting rules between scalars and vectors, which we are aware of from `numpy` or `pytorch`. However, they do not follow the rule of __matching between two `instant vectors`.__: 

## Vector Matching

> Vector matching defines how one `instant vector` can be matched to another.

### One-to-one

> One-to-one matches a unique pair of entries from each side of the operation.

Here are the governing rules:
- The set of labels must be the same.
- The value types must match.

In [ ]:
<vector expr> <op> ignoring(<label list>) <vector expr>
<vector expr> <op> on(<label list>) <vector expr>

- __ignoring__ allows us to ignore label(s).
- __on__ allows us to specify labels.

Assuming that we have the following two groups of time series:
- `method_code:http_errors:rate5m`: `5m` rate of `http_errors` and their specific `code`.
- `method:http_requests:rate5m`: `5m` rate of `http_errors` for the specific method.

Now, for specific `instant selectors`, we might have the following (example values at a given timestamp are commented out on the right):

In [ ]:
method_code:http_errors:rate5m{method="get", code="500"}  # 24
method_code:http_errors:rate5m{method="get", code="404"}  # 30
method_code:http_errors:rate5m{method="put", code="501"}  # 3
method_code:http_errors:rate5m{method="post", code="500"} # 6
method_code:http_errors:rate5m{method="post", code="404"} # 21

method:http_requests:rate5m{method="get"}  # 600
method:http_requests:rate5m{method="del"}  # 34
method:http_requests:rate5m{method="post"} # 120

To determine the ratio of failed requests, we apply `code=500` (internal server error rate):

In [ ]:
method_code:http_errors:rate5m{code="500"} / ignoring(code) method:http_requests:rate5m

This would match
- `method_code:http_errors:rate5m{method="get", code="500"}` with `method:http_requests:rate5m{method="get"} ` (notice that the labels match, while the codes do not. However, we are __ignoring it__), returning the value, `24 / 600 = 0.04`.
- `method_code:http_errors:rate5m{method="post", code="500"}` with `method:http_requests:rate5m{method="post"} ` (same thing as above), returning the value, `6 / 120 = 0.05`.

Therefore, we would obtain two `instant vectors`. 

### Many-to-one and one-to-many

> __Each vector element on the 'one' side can match multiple elements on the 'many' side.__

This use case is advanced and given as a __mandatory__ exercise.

__Note:__ This approach should be used only when necessary.

## Aggregation Operations

> Prometheus provides basic operations for data aggregation

Here is a full list:

- `sum`: calculate sum over dimensions.
- `min`: select minimum over dimensions.
- `max`: select maximum over dimensions.
- `avg`: calculate the average over dimensions.
- `group`: all values in the resulting vector are 1.
- `stddev`: calculate the population standard deviation over dimensions.
- `stdvar`: calculate the population standard variance over dimensions.
- `count`: count the number of elements in the vector.
- `count_values`: count the number of elements with the same value.
- `bottomk`: the smallest k elements based on the sample value.
- `topk`: the largest k elements based on the sample value.
- `quantile`: calculate φ-quantile (0 ≤ φ ≤ 1) over dimensions.

The simplest form is as follows:

```
<op>(<vector_expression>)
```


In [ ]:
min(node_cpu_scaling_frequency_hertz)

In [ ]:
min(windows_cpu_core_frequency_mhz)

![](images/prometheus_op_simple.png)

In this case, the __minimum value for each timestep is taken across all the labels__.

There are two modifiers for specifying the labels (dimensions) across which the operation would run:
- `by`: specifies __by which label(s) the `min` is taken__ (all unspecified will be 'flattened').
- `without`: specifies __without which label(s) the `min` is taken__ (all specified will be 'flattened').

The syntax is as follows:

```
<op> by (<label>) (<vector_expression>)
```

or:

```
<op> without (<label>) (<vector_expression>)
```

The `min` taken for each `cpu` separately would be

In [ ]:
min by (cpu) (node_cpu_scaling_frequency_hertz)

In [ ]:
min by (core) (windows_cpu_core_frequency_mhz)

![](images/prometheus_op_by.png)

## Functions

> __Prometheus provides a set of functions. The full list can be viewed [here](https://prometheus.io/docs/prometheus/latest/querying/functions/).__

Currently, this set is slightly limited, __and it is impossible to add new functions__ (at least without forking the `prometheus` project).

*Things to note*
- The standard rules apply (e.g. some functions have default arguments).
- Some operate on range vectors, while others operate on instant vectors.

Here are some groups of functions:
- `date` related (`minute`, `month`, `year`, `timestamp`, `day_of_month`, etc.)
- `math` related (`ln`, `exp`, `deriv`, `round`, `sgn`, etc.)
- `timeseries` related (`delta` (difference between consecutive values), `idelta`, etc.)

Below are a few examples, with the descriptions provided as comments:

In [2]:
# Monitored hardware temperature difference of `10m` intervals.
# First and last values from the intervals will be taken.

delta(node_hwmon_temp_celsius[10m])

bash: syntax error near unexpected token `node_hwmon_temp_celsius[10m]'


: 2

In [ ]:
delta(windows_net_packets_received_total[10m])

![](images/prometheus_temp_delta.png)

In [ ]:
# Calculates the increase between the last value in the range vector and the first.
# Adjusted for monotonicity and smoothed out.

increase(node_hwmon_temp_celsius[10m])

In [ ]:
increase(windows_net_packets_received_total[10m])

![](images/prometheus_increase_function.png)

In [ ]:
# e^temp exponential value
# This one operates on instant vectors

exp(node_hwmon_temp_celsius)

In [ ]:
exp(windows_cpu_time_total)

![](images/prometheus_exp_function.png)

### {op}_over_time

The last set of functions provided by Prometheus is given by the following scheme:

```
{op}_over_time(v range-vector)
```

Here, different `op`s, such as `avg` and `min`, are introduced, and the operation is carried across specified `range`s.

Check out all the possibilities [here](https://prometheus.io/docs/prometheus/latest/querying/functions/#aggregation_over_time), and consider the example below:

In [ ]:
stddev_over_time(windows_system_context_switches_total[10s])

![](images/prometheus_stddev_over_time.png)

## Recording Rules

> __Recording rules allow us to precompute frequently used/expensive expressions, and save the results as a new timeseries.__

__Always attempt to create new rules instead of running ad-hoc commands__ for the following reasons:
- They are faster, as they work on less data, although regularly.
- They are more readable to others.
- They are easy to reuse.
- They are easy to put in VCSs, such as `git`.

Conventionally, rules are written in a separate `.yml` file (`<name>.rules.yml` appears to be a reasonable choice) __and included in the `prometheus.yml` config__ (discussed in a previous lesson).

Consider the below `example.rules.yml` file:

In [ ]:
groups: # High-level grouping
  - name: example # Name of the group of rules
    interval: 30s # How often they should be evaluated (deafult: 1m)
    rules: # Set of rules in this group
    - record: job:http_inprogress_requests:sum # Name of the rule
      expr: sum by (job) (http_inprogress_requests) # Evaluated expression

### Groups

> Rules within a group are run sequentially (as defined) in a regular interval.

They are grouped based on semantic meaning and evaluation interval.

### Naming

There are a few notable naming guidelines:
- Recording rules should be of the general form, `level:metric:operations`:
    - `level`: labels of the rule output/aggregation level.
    - `metric`: name of the metric, e.g. `http_requests`.
    - `operations`: key operations creating the result.
    
Here are some examples:

In [ ]:
- record: instance_path:requests:rate5m
  expr: rate(requests_total{job="myjob"}[5m])

- record: path:requests:rate5m
  expr: sum without (instance)(instance_path:requests:rate5m{job="myjob"})
  
- record: instance_path:request_failures:rate5m
  expr: rate(request_failures_total{job="myjob"}[5m])

- record:  wmiexporter:windows_cpu_dpcs_total:sum 
  expr: sum by (job) (windows_cpu_dpcs_total)  

### Including rules in prometheus.yml

After the rule is written down, __it must be included in the `prometheus.yml` server config__.

There are two places where one can change the related `recording rules` settings:

In [ ]:
global:
  # How frequently to evaluate rules.
  # Define on a per-group basis if required.
  [ evaluation_interval: <duration> | default = 1m ]
rule_files:
  [ - <filepath_glob> ... ] # Path to rule files 

Consider the below example:

In [ ]:
# my global config
global:
  evaluation_interval: 5m # Evaluate rules every 15 s. The default is every 1 m.

# Load rules once, and periodically evaluate them according to the global 'evaluation_interval'.
rule_files:
   - "windows.rules.yml"
   - "docker.rules.yml"

...

After these steps, your rules should run automatically and be available under your specified `record` name.

## Conclusion
At this point, you should have a good understanding of
- the significant querying capabilities of Prometheus.
- data types.
- instant and range vector selectors.
- vector matching, operators and functions.
- how to record rules.